In [1]:
from PIL import Image;
import numpy as np;
import operator
import matplotlib.pyplot as plt;
import random;
import math;
import statistics

In [2]:
D = (255**2)*3;

def dist(PIX_A, PIX_B):
    d = (PIX_A[0] - PIX_B[0])**2 + (PIX_A[1] - PIX_B[1])**2 + (PIX_A[2] - PIX_B[2])**2;
    return math.sqrt(d/D);

In [3]:
TC = "004"
Folder = "001"

In [4]:
# source_dir = "Source_Images/source_" + TC + ".JPG"
# source_image = Image.open(source_dir)
# source_pix = source_image.load()

source_dir = "Source/"+Folder+"/source_" + TC + ".JPG"
source_image = Image.open(source_dir)
source_pix = source_image.load()

In [5]:
# test_dir = "Copied_Images/copy_" + TC + ".JPG"
# test_image = Image.open(test_dir)
# test_pix = test_image.load()

test_dir = "Copy/"+Folder+"/copy_" + TC + ".JPG"
test_image = Image.open(test_dir)
test_pix = test_image.load()

In [6]:
source_image.size, test_image.size

((710, 838), (286, 154))

In [7]:
spread = 5;

# matrix = np.zeros((test_image.size[0], test_image.size[1]))
start_points = []

for i in range(0, test_image.size[0], 1):
    for j in range(0, test_image.size[1], 1):
        dissimilarity = 0;
        ct = 0;
        for k in range(i-spread, i+spread+1, 1):
            for l in range(j-spread, j+spread+1, 1):
                if(k == i and j == l):
                    continue;
                if(k < 0 or k >= test_image.size[0] or l < 0 or l >= test_image.size[1]):
                    continue;
                dissimilarity += dist(test_pix[k, l], test_pix[i, j]);
        
#         Diss = dissimilarity/ct;
#         matrix[i][j] = dissimilarity * statistics.variance(test_pix[i, j]);
        Diss = dissimilarity ;
        Diss = round(Diss, 2)
        start_points.append([Diss, i, j]);

start_points = sorted(start_points, key = operator.itemgetter(0));
start_points.reverse()

In [8]:
start_points[0]

[106.34, 75, 116]

In [9]:
len(start_points)

44044

In [10]:
PTS = 100;
D = 5;

for i in range(0, PTS, 1):
    X = start_points[i][1];
    Y = start_points[i][2];
    
#     test_pix[X, Y] = (0, 0, 0);
    for j in range(X - D, X + D + 1, 1):
        for k in range(Y - D, Y + D + 1, 1):
            test_pix[j, k] = (0, 0, 0);
    
test_image.save("W.jpg")

In [11]:
# LMT = 100;
# x = np.arange(0, LMT, 1)
# y = [];
# for i in range(0, LMT, 1):
#     y.append(start_points[i][0])
# plt.plot(x, y)

In [12]:
# start_points = start_points[:100]
# len(start_points)

In [13]:
# fig = plt.figure(figsize = (10,10)) 
# plt.imshow( matrix , cmap = 'copper' , interpolation = 'nearest')
# plt.show()

In [14]:
class Path:
    def __init__(self):
        self.points = [];
        self.score = [];
        self.agg_score = -1;

In [15]:
def aggregate_score(scores):
    if(len(scores) == 0): return 0;
    if(scores[-1] == -1): return -1;
    random.shuffle(scores);
    total_buckets = 100;
    max_bucket_size = len(scores)/total_buckets;
    buckets = [];
    curr_size = 0;
    curr_sum = 0;
    for sc in scores:
        curr_size += 1;
        curr_sum += sc;
        if(curr_size == max_bucket_size):
            buckets.append(curr_sum/curr_size);
            curr_sum = 0;
            curr_size = 0;
    if(curr_size > 0):
        buckets.append(curr_sum/curr_size);
    
    return statistics.median(buckets);

In [16]:
ans = Path();
ans.agg_score = -1;

In [17]:
MAX_STEP = 10;
MAX_STEP = math.floor(min(test_image.size[0], test_image.size[1])/10);

In [19]:
MAX_STEP

15

In [20]:
len(start_points)

44044

In [21]:
def centroid_calculation(pts):
    n = len(pts);
    if(n == 0): return [0, 0];
    
    x_sum, y_sum = 0, 0;
    for pt in pts:
        x_sum += pt[0];
        y_sum += pt[1];
    
    x_c = round(x_sum/n, 2);
    y_c = round(y_sum/n, 2);
    
    return [x_c, y_c];

In [22]:
# pts = [[0, 1], [5, 6], [12, 23], [78, 19]]
# centroid_calculation(pts)

In [30]:
RANDOM_WALK_LEN = 100;

In [31]:
ITR_LIMIT = 100;

paths = [];
for i in range(0, 100, 1):
    p = Path();
    paths.append(p);


for START in range(0, ITR_LIMIT, 1):
    print("ITR: ", START+1);
#     print(paths[0].score)
    start_X = start_points[START][1];
    start_Y = start_points[START][2];

    v = []
    for i in range(0, source_image.size[0], 1):
        for j in range(0, source_image.size[1], 1):
            d = dist(test_pix[start_X, start_Y], source_pix[i,j]);
            v.append([d, i, j]);

    v = sorted(v, key = operator.itemgetter(0));
    v = v[:100]
    
    paths = sorted(paths, key=lambda x: aggregate_score(x.score), reverse=True);
    
    visited = [False] * len(v);
    for i in range(0, len(paths), 1):
        if(len(paths[i].score) > 0 and paths[i].score[-1] == -1):
            continue;
            
        cen = centroid_calculation(paths[i].points);
        
        min_dis = -1;
        min_dis_id = -1;
        
        for j in range(0, len(v), 1):
            if(visited[j] == True): continue;
            D = (v[j][1] - cen[0])**2 + (v[j][2] - cen[1])**2;
            if(min_dis == -1 or min_dis > D):
                min_dis = D;
                min_dis_id = j;
        
        paths[i].points.append([v[min_dis_id][1], v[min_dis_id][2]]);
        paths[i].score.append(v[min_dis_id][0]);
        visited[min_dis_id] = True;
    
#     for i in range(0, len(v), 1):
#         if(len(paths[i].score) > 0 and paths[i].score[-1] == -1):
#             continue;
#         paths[i].points.append([v[i][1], v[i][2]]);
#         paths[i].score.append(v[i][0]);
    
    curr_X = start_X;
    curr_Y = start_Y;

    for itr in range(0, RANDOM_WALK_LEN, 1):
        step_X = random.randint(1, MAX_STEP);
        step_Y = random.randint(1, MAX_STEP);
        
        off_X = random.randint(0, 1);
        off_Y = random.randint(0, 1);

        if(off_X == 0): off_X = -1;
        if(off_Y == 0): off_Y = -1;

        curr_X = curr_X + off_X * step_X;
        curr_Y = curr_Y + off_Y * step_Y;

        if(curr_X < 0 or curr_X >= test_image.size[0] or curr_Y < 0 or curr_Y >= test_image.size[1]):
            curr_X = curr_X - off_X * step_X;
            curr_Y = curr_Y - off_Y * step_Y;
            continue;

        img_RGB = test_pix[curr_X, curr_Y];

        for i in range(0, len(paths), 1):
            if(paths[i].score[-1] == -1):continue;
            last_X = paths[i].points[-1][0];
            last_Y = paths[i].points[-1][1];

            new_X = last_X + off_X * step_X;
            new_Y = last_Y + off_Y * step_Y;

            if(new_X < 0 or new_X >= source_image.size[0] or new_Y == 0 or new_Y >= source_image.size[1]):
                paths[i].score.append(-1);
                continue;

            d = dist(img_RGB, source_pix[new_X, new_Y]);

            paths[i].score.append(d);
            paths[i].points.append([new_X, new_Y]);

ITR:  1
ITR:  2
ITR:  3
ITR:  4
ITR:  5
ITR:  6
ITR:  7
ITR:  8
ITR:  9
ITR:  10
ITR:  11
ITR:  12
ITR:  13
ITR:  14
ITR:  15
ITR:  16
ITR:  17
ITR:  18
ITR:  19
ITR:  20
ITR:  21
ITR:  22
ITR:  23
ITR:  24
ITR:  25
ITR:  26
ITR:  27
ITR:  28
ITR:  29
ITR:  30
ITR:  31
ITR:  32
ITR:  33
ITR:  34
ITR:  35
ITR:  36
ITR:  37
ITR:  38
ITR:  39
ITR:  40
ITR:  41
ITR:  42
ITR:  43
ITR:  44
ITR:  45
ITR:  46
ITR:  47
ITR:  48
ITR:  49
ITR:  50
ITR:  51
ITR:  52
ITR:  53
ITR:  54
ITR:  55
ITR:  56
ITR:  57
ITR:  58
ITR:  59
ITR:  60
ITR:  61
ITR:  62
ITR:  63
ITR:  64
ITR:  65
ITR:  66
ITR:  67
ITR:  68
ITR:  69
ITR:  70
ITR:  71
ITR:  72
ITR:  73
ITR:  74
ITR:  75
ITR:  76
ITR:  77
ITR:  78
ITR:  79
ITR:  80
ITR:  81
ITR:  82
ITR:  83
ITR:  84
ITR:  85
ITR:  86
ITR:  87
ITR:  88
ITR:  89
ITR:  90
ITR:  91
ITR:  92
ITR:  93
ITR:  94
ITR:  95
ITR:  96
ITR:  97
ITR:  98
ITR:  99
ITR:  100


In [32]:
for p in paths:
    if(p.score[-1] == -1):
        continue;

    p.agg_score = aggregate_score(p.score);
    if(p.agg_score < ans.agg_score or ans.agg_score == -1):
        ans = p;

In [33]:
len(paths)

100

In [34]:
# LMT = 100;
# x = np.arange(0, LMT, 1)
# y = [];
# for i in range(0, LMT, 1):
#     y.append(v[i][0])
# plt.plot(x, y)

In [27]:
# output_dir = "Results/result_" + TC + ".jpg";
# for i in range(0, len(ans.points), 1):
# #     print(final_P.points[i]);
#     source_pix[ans.points[i][0], ans.points[i][1]] = (0, 0, 0)

# source_image.save(output_dir)

output_dir = "New_Results/" + Folder + "/result_"+ TC + ".jpg";
for i in range(0, len(ans.points), 1):
#     print(final_P.points[i]);
    source_pix[ans.points[i][0], ans.points[i][1]] = (0, 0, 0)

source_image.save(output_dir)

In [28]:
ans.agg_score

0.418291536215994

In [29]:
1 - ans.agg_score

0.581708463784006